In [1]:
import torch
import utils
import imp
import numpy as np
from torch import nn
from trainer import concrete_trainer
from torch.utils import data
from network import ConcreteAutoencoder
%matplotlib inline

/home/qinxianhan/.conda/envs/singleCell/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_703456/3715685474.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
imp.reload(utils)

In [2]:
file_path = "../../data/filtered_gene_bc_matrices/hg19/"
device = utils.get_device()
#device = torch.device("cpu")
device

device(type='cuda', index=1)

In [3]:
adata = utils.anndata_load(file_path)
adata = utils.anndata_preprocess(adata)
adata.shape

(2700, 4758)

In [4]:
target_index = adata.var_names.get_loc("XBP1")
target_index

4593

In [5]:

tf_adata = utils.get_TF("../../data/tf/TF_uniq.txt", adata, target_index)
tf_adata.shape
tf_adata[:,-1]

[1343, 2780, 2574, 2938, 453, 2891, 2000, 3297, 1293, 821, 1547, 4333, 3611, 3732, 3636, 1739, 2331, 4153, 2127, 557, 1320, 3750, 2860, 768, 2259, 3517, 2424, 3642, 881, 1849, 4481, 3915, 3638, 4437, 3131, 1206, 2841, 2842, 3296, 959, 2394, 3525, 227, 4594, 1863, 119, 1647, 1381, 1, 3268, 4722, 1617, 4416, 2215, 65, 1779, 1340, 1236, 4496, 3956, 200, 4309, 4358, 2724, 3600, 2230, 2846, 2175, 1122, 2302, 2828, 1190, 4317, 4126, 3274, 3576, 1009, 2058, 1314, 2192, 3766, 735, 4231, 2505, 154, 1260, 1383, 2942, 1994, 4116, 3421, 3427, 3742, 4446, 2128, 2182, 3832, 3145, 4054, 3615, 1120, 2723, 1905, 1140, 3400, 4714, 76, 2360, 4033, 3419, 4295, 4583, 895, 2970, 2520, 4264, 3417, 727, 2635, 4505, 745, 2976, 3843, 746, 3842, 3841, 2981, 762, 3939, 3127, 2393, 1379, 1051, 3868, 2146, 3178, 2123, 3645, 4127, 4552, 3489, 3526, 3857, 4393, 3895, 4593, 4090, 3330, 4236, 673, 1831, 3514, 2174, 1787, 3566, 4593]


View of AnnData object with n_obs × n_vars = 2700 × 1
    obs: 'n_genes'
    var: 'gene_ids', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [6]:
def get_net(input_dim, k, hidden_dim,device):
    return ConcreteAutoencoder(input_dim, k,hidden_dim, out_dim = 1,device = device)
def load_adata(adata, batch_size):
    return utils.data_loader(adata, batch_size)

In [7]:
num_epoch, learning_rate, weight_dacay, batch_size = 256, 0.05, 0, 256
input_dim = tf_adata.n_vars - 1
init_temp = 10
end_temp = 0.01
loss = nn.MSELoss()
net = get_net(input_dim=input_dim, k=10, hidden_dim=128, device=device)
dataloader = load_adata(adata=tf_adata, batch_size=batch_size)

In [ ]:
concrete_trainer(net=net, loss=loss, dataloader=dataloader,
      num_epoch=num_epoch, learning_rate=learning_rate,
      weight_decay=weight_dacay ,device=device, init_temp=init_temp, 
      end_temp= end_temp)

In [ ]:
x_val = torch.Tensor(tf_adata.X.A[:,0:-1]).to(device) # type: ignore
_, z, m = net(x_val, temperature = end_temp)
z.shape

In [ ]:
z = z.to('cpu').detach().numpy()
m = m.to('cpu').detach().numpy()

In [ ]:
selected_tf = np.argmax(m,axis=0)

print(adata.var_names[target_index])
print (tf_adata.var_names[selected_tf])

In [9]:
selected_tf = np.zeros([10,10])
for i in range(10):
    net = get_net(input_dim=input_dim, k=10, hidden_dim=128, device=device)
    dataloader = load_adata(adata=tf_adata, batch_size=batch_size)
    concrete_trainer(net=net, loss=loss, dataloader=dataloader,
      num_epoch=num_epoch, learning_rate=learning_rate,
      weight_decay=weight_dacay ,device=device, init_temp=init_temp, 
      end_temp= end_temp)
    x_val = torch.Tensor(tf_adata.X.A[:,0:-1]).to(device)
    _, z, m = net(x_val, temperature = end_temp)
    m = m.to('cpu').detach().numpy()
    selected_tf[i] = np.argmax(m,axis=0)

Epoch: 1 ; Loss: 0.86480; Temperature: 9.734; Time: 0.02 s
Epoch: 2 ; Loss: 0.80423; Temperature: 9.475; Time: 0.05 s
Epoch: 3 ; Loss: 0.80206; Temperature: 9.222; Time: 0.07 s
Epoch: 4 ; Loss: 0.79511; Temperature: 8.977; Time: 0.09 s
Epoch: 5 ; Loss: 0.79372; Temperature: 8.738; Time: 0.12 s
Epoch: 6 ; Loss: 0.79424; Temperature: 8.505; Time: 0.14 s
Epoch: 7 ; Loss: 0.79076; Temperature: 8.279; Time: 0.16 s
Epoch: 8 ; Loss: 0.79943; Temperature: 8.058; Time: 0.19 s
Epoch: 9 ; Loss: 0.79551; Temperature: 7.844; Time: 0.21 s
Epoch: 10 ; Loss: 0.79599; Temperature: 7.635; Time: 0.23 s
Epoch: 11 ; Loss: 0.80133; Temperature: 7.432; Time: 0.25 s
Epoch: 12 ; Loss: 0.79778; Temperature: 7.234; Time: 0.28 s
Epoch: 13 ; Loss: 0.79103; Temperature: 7.041; Time: 0.30 s
Epoch: 14 ; Loss: 0.79841; Temperature: 6.854; Time: 0.32 s
Epoch: 15 ; Loss: 0.79398; Temperature: 6.671; Time: 0.35 s
Epoch: 16 ; Loss: 0.79133; Temperature: 6.494; Time: 0.37 s
Epoch: 17 ; Loss: 0.79757; Temperature: 6.321; Ti

In [ ]:
selected_tf

In [10]:
idx, freq = np.unique(selected_tf, return_counts=True)
idx, freq

(array([  0.,   1.,   5.,   6.,   7.,  13.,  14.,  17.,  19.,  21.,  23.,
         24.,  33.,  34.,  41.,  46.,  48.,  49.,  53.,  54.,  55.,  59.,
         64.,  66.,  68.,  73.,  75.,  76.,  77.,  78.,  79.,  80.,  85.,
         86.,  90.,  92.,  96., 100., 102., 103., 104., 105., 108., 109.,
        112., 113., 114., 117., 119., 120., 121., 122., 123., 126., 130.,
        132., 134., 135., 137., 138., 141., 142., 143., 148., 149., 151.,
        153., 154.]),
 array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, 5, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 5, 2, 1, 1,
        1, 3, 2, 3, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1,
        1, 1]))

In [11]:
idx = idx[np.argsort(freq)][::-1].astype(int)
tf_adata.var_names[idx]

Index(['HES4', 'RORA', 'SP3', 'CREB3', 'STAT6', 'MAZ', 'SMARCC2', 'REPIN1',
       'SMC3', 'RUNX1', 'NR3C1', 'PIAS1', 'STAT2', 'MBD4', 'ELF1', 'CREB1',
       'MECP2', 'AFF4', 'UBTF', 'UBN1', 'ZBTB7A', 'STAT1', 'KDM5A', 'JMJD6',
       'IKZF1', 'ID3', 'HSF1', 'HIF1A', 'USF2', 'HDAC2', 'GLYR1', 'EGLN2',
       'ZC3H8', 'ZNF263', 'CHD3', 'CEBPZ', 'CEBPB', 'CBFB', 'C17orf49', 'BATF',
       'ATRX', 'ATF7IP', 'ARRB1', 'KAT8', 'KLF3', 'SPIB', 'TFDP1', 'STAT3',
       'STAT4', 'TAF3', 'SMARCC1', 'SMAD3', 'SMAD2', 'TBL1XR1', 'TERF1',
       'REST', 'RBPJ', 'MAFB', 'RARA', 'POLR2A', 'THAP11', 'ZNF92', 'NIPBL',
       'NCOR1', 'MYC', 'MEF2C', 'TOP1', 'CCDC101'],
      dtype='object')

In [ ]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc

In [ ]:
res_adata = ad.AnnData(z)

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')


In [ ]:
sc.pl.pca(adata)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)

In [ ]:
sc.pl.umap(adata, color='RBL2')

In [ ]:
max_idx = []
for i in range(m.shape[1]):
    max_idx.append(np.argmax(m[:,i]))
max_idx

In [ ]:
for i in range(m.shape[1]):
    print(max(m[:,i]))

In [ ]:
def find_same(index):
    count = 0
    for i in range(1,len(index)):
        for j in range(i):
            if index[j] == index[i]:
                count +=1
    print(count)  

In [ ]:
find_same(max_idx)

In [ ]:
res_genes = adata.var_names[max_idx]

In [ ]:
res_adata.var_names = res_genes # type: ignore

In [ ]:
sc.tl.rank_genes_groups(res_adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(res_adata, n_genes=25, sharey=False)

In [ ]:
adata = utils.anndata_load(file_path=file_path)
adata = utils.anndata_preprocess(adata=adata,n_top_genes=1000)

In [ ]:
var_genes = adata.var_names

In [ ]:
res_adata.var_names_make_unique()

In [ ]:
s = set(res_adata.var_names)

In [ ]:
temp1 = [x for x in var_genes if x not in s]

In [ ]:
len(temp1)

In [ ]:
temp2 = [x for x in s if x not in var_genes]

In [ ]:
len(temp2)

In [ ]:
temp1

In [ ]:
temp2